# Análise exploratória de dados espaciais

<br>
Municípios grandes produtores de alguma cultura tendem a estar próximos de municípios também produtores dessa cultura ou de municípios que não o produzem? O objetivo da análise exploratória de dados espaciais (AEDE) é responder a esse tipo de questão. De forma mais geral, a questão é qual a a associação entre o valor de certa variável em um lugar e os valores dessa mesma variável nos lugares vizinhos. Localidades com valores semelhantes de determinada variável tendem a estar próximas? Para isso, ela mede a autocorrelação espacial, por meio do $I$ de Moran.

In [0]:
# instalação dos pacotes necessários
!sudo apt install python3-rtree   # solução para o problema
!pip install geopandas pysal mapclassify

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gp
import pysal as ps
import seaborn as sns
from pysal.lib.weights.contiguity import Queen
from pysal.explore.esda import Moran, Moran_Local
from pysal.viz.splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation
from pysal.viz import mapclassify
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/'
link_p = 'https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/'

In [0]:
# ler dados de interesse
dados = pd.read_csv(link_p+'qtde-22.csv', encoding='latin1')
# quantidade produzida em toneladas em 2017

In [0]:
# para ler um arquivo salvo no computador
# ir primeiro em > e Files
# dados = pd.read_csv('qtde_22.csv', encoding='latin1')

In [0]:
# nomes das variáveis


In [0]:
# número de linhas e colunas do dataframe


In [0]:
# mostrar algumas linhas do dataframe


In [0]:
# renomear colunas
# 'ibge7' será 'mun' (código do município)
# 'mun' será 'município' (nome do município)

dados.rename(columns={'mun': 'municipio'}, inplace=True)  # inplace = True modifica o dataframe
dados.rename(columns={'ibge7': 'mun'}, inplace=True)

In [0]:
# ler shapefile
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/mg.json'
geodf = gp.read_file(link)

In [0]:
# visualizar as primeiras linhas do shapefile


In [0]:
# mesclar shapefile com dataframe
# usando a coluna mun
geodf.rename(columns={'CD_GEOCMU': 'mun'}, inplace=True)  # mudar nome da coluna com cód. para 'mun'
geodf['mun'] = geodf.mun.astype(int)  # transformar códigos em inteiros
mg = pd.merge(geodf, dados, on='mun', suffixes=('', '_y'))  # mesclar o dataframe e o shapefile
mg.shape

In [0]:
# visualizar primeiras linhas objeto resultante (dataframe + shapefile)


In [0]:
# colunas (variáveis) do objeto resultante


In [0]:
# escolher uma variável dos dados
variavel = 'batata-inglesa'

In [0]:
# mapa temático - simples
mg.plot(variavel);

Esquemas de cores: cmap (https://matplotlib.org/tutorials/colors/colormaps.html)

- PuBu

- GnBu

- Oranges

- BuGn

- Purples

- YlOrBr


In [0]:
# mapa temático - com opções
# scheme='Quantiles' ou 'Equal_Interval'
# quantiles: atribui mesma quantidade de valores para cada categoria. Apesar de esconder valores extremos,
# pode se tornar mais informativa se a distribuição for assimétrica
# perceber as diferenças entre os esquemas: cmap='PuBu', por exemplo


mg.plot(column=variavel, figsize=(13, 13), scheme='quantiles', legend=True, k=4);

In [0]:
# visualizar número de observações em cada faixa de valores
classi = ps.viz.mapclassify.Equal_Interval(mg[variavel], k=6)
classi

In [0]:
# histograma suavizado
f, ax = plt.subplots(1) # define a figura
sns.kdeplot(mg[variavel], shade=True) # plota estimação de densidade Kernel (KDE)
sns.rugplot(mg[variavel], alpha=0.5) # adiciona um risco azul para cada valor na base do gráfico
for cut in classi.bins:
  plt.axvline(cut, color='red', linewidth=0.75);  # plota uma linha vermelha nas quebras dos intervalos definidos

In [0]:
# mapa temático - com opções
# escolher diferentes esquemas de cores
mg.plot(column=variavel, figsize=(13, 13), scheme='equal_interval', legend=True, k=4);

In [0]:
# mapa temático - com opções
# scheme='equal_interval', 'fisher_jenks'
mg.plot(column=variavel, figsize=(13, 13), scheme='fisher_jenks', legend=True, k=4);

In [0]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(mg)
w.transform = 'r'

### I de Moran

Estatística mais utilizada para medir a autocorrelação espacial. Ela mede a relação do desvio padronizado de uma variável numa área com o desvio padronizado das áreas vizinhas para a mesma variável:

$$I=\frac{N}{S_{0}}\frac{\displaystyle\sum_{i=1}^{n}\sum_{j=1}^{n}w_{ij}z_{i}z_{j}}{\displaystyle\sum_{i=1}^{n}z^{2}_{i}},$$
em que:

- $z_i = y_i - \bar{y}$ representa o desvio em relação à média da variável.
- $y_i$: valor da variável em um determinado local $i$
- $N$: número de observações
- $S_0 = \sum\sum w_{ij}$
- $E[I] = -1/(N - 1) \approx 0$
            
Se o valor-$p$ referente ao teste do $I$ de Moran for significativo, podemos olhar para o valor da estatística $I$ e concluir:

- $I > 0$: autocorrelação espacial positiva (*clusters* espaciais - HH, LL)
- $I < 0$: autocorrelação espacial negativa (*outliers* espaciais - HL, LH)

In [0]:
# calcular I de Moran global para a variável escolhida
y = mg[variavel].values
moran = Moran(y, w)
moran.I

In [0]:
# valor-p
moran.p_sim

In [0]:
# diagrama de dispersão de Moran
plot_moran(moran, zstandard=False, figsize=(10,4));

In [0]:
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

### LISA

- Estatística para detectar padrões locais de autocorrelação espacial: *Local Indicator of Spatial Association* (LISA), ou $I$ de Moran local  
- Permite verificar se há agrupamentos espaciais estatisticamente significativos
- Útil quando a estatística $I$ de Moran global for significativa

$$I_{i}=z_{i}\sum_{j=1}^{j}w_{ij}y_{j}$$  

In [0]:
lisa_cluster(moran_loc, mg, p=0.05, figsize = (9,9));

In [0]:
plot_local_autocorrelation(moran_loc, mg, variavel);